In [1]:
import nltk
import re
import pandas as pd


In [37]:
# this is the test part for NLTK

text = 'Then the limitations of the power supply have to be checked.'
sents = nltk.sent_tokenize(text)


In [38]:
sents

['Then the limitations of the power supply have to be checked.']

In [39]:
words = [nltk.word_tokenize(sentences) for sentences in sents]

In [40]:
words

[['Then',
  'the',
  'limitations',
  'of',
  'the',
  'power',
  'supply',
  'have',
  'to',
  'be',
  'checked',
  '.']]

In [41]:
tags = [nltk.pos_tag(tokens) for tokens in words]

In [42]:
tags

[[('Then', 'RB'),
  ('the', 'DT'),
  ('limitations', 'NNS'),
  ('of', 'IN'),
  ('the', 'DT'),
  ('power', 'NN'),
  ('supply', 'NN'),
  ('have', 'VBP'),
  ('to', 'TO'),
  ('be', 'VB'),
  ('checked', 'VBN'),
  ('.', '.')]]

In [ ]:
# tries 


# 1. Named Entity Recognition--NER

In [8]:
from nltk.tokenize import PunktSentenceTokenizer

In [9]:
f = open('motor_design1.txt',encoding='utf-8')
raw = f.read()     
#print (type(raw))   #classs 'str'
print (raw)

A1) What you do to design a simple motor. (english )

For a simple motor at least the rated power, rated speed, the supply
voltage and the maximum current should be given.

The rated torque is calculated from power and speed, Tn = Pn/ nn *
30/PI.

For a simple motor with good efficiency a radial flux permanent magnet
synchronous machine (PMSM) will be chosen, but the specification may
require a different machine type like axial flux PMSM, separately
excited synchronous, asynchronous or reluctance machine.

Depending on the application also an “inner rotor” or “outer rotor”
topology has to be chosen in case of a radial flux PMSM.

If the machine size is not defined it has to be calculated from the
physical limitation factors.

The bore diameter and axial length need to be chosen to meet the mean
shear stress requirements. Depending on the rated power an approximate
mean shear stress value can be looked up from literature e.g. sigma =
12 to 300 kN/m2 for P= 0.01 to 160 MVA.

e.g. length 

In [10]:
sample_text = raw
custom_sent_tokenizer = PunktSentenceTokenizer()
tokenized = custom_sent_tokenizer.tokenize(sample_text)

In [11]:
# generate the entity tree of the text

def process_content():
    try:
        for i in tokenized[5:]:
            words = nltk.word_tokenize(i)
            tagged = nltk.pos_tag(words)
            namedEnt = nltk.ne_chunk(tagged, binary=True)
            namedEnt.draw()
    except Exception as e:
        print(str(e))

In [12]:
process_content() # only launch it when you want to see the analysis results for every sentences. I still can't find a proper way to make a output with reasonalbe number.

In [13]:
def parse_document(document):
   document = re.sub('\n', ' ', document)
   if isinstance(document, str):
       document = document
   else:
       raise ValueError('Document is not string!')
   document = document.strip()
   sentences = nltk.sent_tokenize(document)
   sentences = [sentence.strip() for sentence in sentences]
   return sentences

# sample document
text = """
SyMSpace MotorBox provides various machine simulation components,
materials and geometries, which can be used to set up a workflow.
"""

# tokenize sentences
sentences = parse_document(text)
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
# tag sentences and use nltk's Named Entity Chunker
tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
ne_chunked_sents = [nltk.ne_chunk(tagged) for tagged in tagged_sentences]
# extract all named entities
named_entities = []
for ne_tagged_sentence in ne_chunked_sents:
   for tagged_tree in ne_tagged_sentence:
       # extract only chunks having NE labels
       if hasattr(tagged_tree, 'label'):
           entity_name = ' '.join(c[0] for c in tagged_tree.leaves()) #get NE name
           entity_type = tagged_tree.label() # get NE category
           named_entities.append((entity_name, entity_type))
           # get unique named entities
           named_entities = list(set(named_entities))

# store named entities in a data frame
entity_frame = pd.DataFrame(named_entities, columns=['Entity Name', 'Entity Type'])
# display results
print(entity_frame)


  Entity Name   Entity Type
0    SyMSpace  ORGANIZATION
1    MotorBox  ORGANIZATION


## First problem remain to be solved
**We must find way to detect those entity in the mortor design process**

In [14]:
from textblob import TextBlob

In [15]:
txt = """For a simple motor with good efficiency a radial flux permanent magnet
synchronous machine (PMSM) will be chosen, but the specification may
require a different machine type like axial flux PMSM, separately
excited synchronous, asynchronous or reluctance machine."""

In [16]:
blob = TextBlob(txt)

In [17]:
print(blob.noun_phrases)

['simple motor', 'good efficiency', 'radial flux', 'permanent magnet synchronous machine', 'pmsm', 'different machine type', 'axial flux', 'pmsm', 'reluctance machine']


# 2. Analysis the meaning of sentences (the logic behind it)

In [18]:
IN = re.compile(r'.*\bin\b(?!\b.+ing)')
for doc in nltk.corpus.ieer.parsed_docs('NYT_19980315'):
    for rel in nltk.sem.extract_rels('ORG', 'LOC', doc,
        corpus='ieer', pattern = IN):
        print(nltk.sem.rtuple(rel))

[ORG: 'WHYY'] 'in' [LOC: 'Philadelphia']
[ORG: 'McGlashan &AMP; Sarrail'] 'firm in' [LOC: 'San Mateo']
[ORG: 'Freedom Forum'] 'in' [LOC: 'Arlington']
[ORG: 'Brookings Institution'] ', the research group in' [LOC: 'Washington']
[ORG: 'Idealab'] ', a self-described business incubator based in' [LOC: 'Los Angeles']
[ORG: 'Open Text'] ', based in' [LOC: 'Waterloo']
[ORG: 'WGBH'] 'in' [LOC: 'Boston']
[ORG: 'Bastille Opera'] 'in' [LOC: 'Paris']
[ORG: 'Omnicom'] 'in' [LOC: 'New York']
[ORG: 'DDB Needham'] 'in' [LOC: 'New York']
[ORG: 'Kaplan Thaler Group'] 'in' [LOC: 'New York']
[ORG: 'BBDO South'] 'in' [LOC: 'Atlanta']
[ORG: 'Georgia-Pacific'] 'in' [LOC: 'Atlanta']


In [19]:
print('I noticed that the big yellow dog(NN) is eating(V) fresh meat(N)')

I noticed that the big yellow dog(NN) is eating(V) fresh meat(N)


# 3. Find Stop words

In [20]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
example_sent = """For a simple motor with good efficiency a radial flux permanent magnet
synchronous machine (PMSM) will be chosen, but the specification may
require a different machine type like axial flux PMSM, separately
excited synchronous, asynchronous or reluctance machine."""
  
stop_words = set(stopwords.words('english')) 
  
word_tokens = word_tokenize(example_sent) 
  
filtered_sentence = [w for w in word_tokens if not w in stop_words] 

filtered_sentence = [] 

for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 

#print(word_tokens) 
#print(filtered_sentence) 

result_sentence = [1]*len(filtered_sentence)

for i in range(len(filtered_sentence)):
    result_sentence[i] = filtered_sentence[i]

#print(result_sentence)
    



# 4. Find ambiguous words (Lesk algorithm)

In [21]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords

In [22]:
stopWords = set(stopwords.words('english'))
processedStopWords = set()
for stopWord in stopWords:
    tokens = nltk.word_tokenize(stopWord)
    for token in tokens:
        processedStopWords.add(token)


In [23]:
def LeskTokenize(s):
    """
    分词工具
    输入: 一个字符串s
    输出: 一个有词构成的列表，不含stop word，数字和标点符号
    """
    global processedStopWords
    result = []
    cleanString = ''
    for c in s:
        if c.isalpha() or c == ' ':
            cleanString = cleanString + c
    rawTokens = nltk.word_tokenize(cleanString)
    for rawToken in rawTokens:
        if rawToken not in processedStopWords:
            result.append(rawToken)
    return result


In [24]:
def overlap(a, b):
    """
    此函数计算两个列表之间有多少相同的元素
    输入: 两个列表a, b
    输出: 两个列表中的相同元素个数，int
    """
    result = 0
    if len(a) == 0 or len(b) == 0:
        return result
    for x in a:
        for y in b:
            if x == y:
                result += 1
    return result


In [25]:
def LeskDisambiguisation(s, w):
    """
    Lesk算法函数
    输入: 1, 一个字符串(句子)s;
         2, 一个字符串（需要消歧义的单词）w;
    输出: 需要消歧义的词的在该句子中的含义;
    """
    tokens = LeskTokenize(s)
    # 特殊情况
    if len(tokens) == 0:
        raise Exception("Sorry, there is nothing in the sentence. ")
    if w not in tokens:
        raise Exception("Sorry, the word is not in the sentence or it is a stop word. ")
    context = set() # 使用集合，避免重复
    for token in tokens:
        if token != w:
            context.add(token)
    if len(context) == 0:
        raise Exception("Sorry, the word that needs desambiguation is the only word in the sentence. ")
    contextDefinitions = []
    for v in context:
        for i in range(len(wn.synsets(v))):
            definitionWords = LeskTokenize(wn.synset(wn.synsets(v)[i].name()).definition())
            for definitionWord in definitionWords:
                contextDefinitions.append(definitionWord)
    maxValue = 0
    maxNumber = 0
    for j in range(len(wn.synsets(w))):
        definitionWords = LeskTokenize(wn.synset(wn.synsets(w)[j].name()).definition())
        currentValue = overlap(definitionWords, contextDefinitions)
        if currentValue > maxValue:
            maxValue = currentValue
            maxNumber = j
    return wn.synset(wn.synsets(w)[maxNumber].name()).definition()


In [30]:
example_sent1 = """For a simple motor at least the rated power, rated speed, the supply
voltage and the maximum current should be given."""

example_sent2 = """I went to the bank to deposit my money"""

example_sent3 = """River bank destroyed by flood"""

LeskDisambiguisation(example_sent1,'current')
#LeskDisambiguisation(example_sent2,'bank')


'a steady flow of a fluid (usually from natural causes)'